In [11]:
# 시세 증가

import requests
from bs4 import BeautifulSoup as bs

kospi_url = "https://finance.naver.com/sise/sise_rise.nhn?sosok=0"
kosdak_url = "https://finance.naver.com/sise/sise_rise.nhn?sosok=1"
list_url = [
    kospi_url, kosdak_url
]

def remake_list(list_base):
    num_cols = [2,3,5,6,7,8,9]
    rate_cols = [4,10,11]
    for idx in range(len(num_cols)):
        try:
            num = int(list_base[num_cols[idx]].replace(",",""))
        except:
            num = 0
        list_base[num_cols[idx]] = num
    for idx in range(len(rate_cols)):
        try:
            num = float(list_base[rate_cols[idx]].replace(",","").replace("%","").replace("+",""))
        except:
            num = 0
        list_base[rate_cols[idx]] = num
        
    return list_base

      
list_whole = []

def get_data(base_url):
    response = requests.get( base_url )
    soup = bs(response.text, 'html.parser')

    content = soup.select("div.box_type_l")
    list_content = str(content).split("\n")
    list_jongmok = []
    for row_data in list_content:
        row = row_data.strip().replace("\t","")
        if '<td class="no">' in row:
            if len(list_jongmok) > 0:
                list_whole.append(list_jongmok)
                list_jongmok = []
        elif "tltle" in row:
            jongmok_cd = row.split("code=")[1].split('"')[0]
            jongmok_nm = row.split("code=")[1].split('"')[1].split('</a')[0].replace(">","")
            list_jongmok.append("A" + jongmok_cd)
            list_jongmok.append(jongmok_nm)
        elif row[:1] != "<":
            list_jongmok.append(row)
        elif ("<td" not in row or row[:4] == "<tr>"):
            continue
        else:
            num_val = row.replace('<td class="number">','')
            if len(num_val) == 0:
                continue
            list_jongmok.append(num_val.replace("</td>",""))


for url in list_url:
    get_data(url)
    
list_result = []
for list_row in list_whole:
    if len(list_row) == 12:
        list_result.append(remake_list(list_row))
        
head = """
insert into naver_sise_rising
(jongmok_cd, jongmok_nm, prc, pre_vs_gap, pre_vs_rt, vol, buy_prc, sell_prc, buy_remain_vol, sell_remain_vol, per, roe)
values
"""
body = ""
for list_row in list_result:
    if list_row[11] < 0.0: continue
    row = "("
    for idx in range(len(list_row)):
        try:
            row += "'" + list_row[idx] + "', "
        except:
            row += str(list_row[idx]) + ", "
    row += ")," + "\n"
    row = row.replace(", )", ")")
    body += row

ins_qry = head + body
print(ins_qry[:len(ins_qry)-2])


insert into naver_vol_rising
(jongmok_cd, jongmok_nm, prc, pre_vs_gap, pre_vs_rt, vol, buy_prc, sell_prc, buy_remain_vol, sell_remain_vol, per, roe)
values
('A007110', '일신석재', 2700, 395, 17.14, 114822689, 2700, 2705, 195392, 430188, 180.0, 1.22),
('A138490', '코오롱플라스틱', 7760, 970, 14.29, 13120413, 7760, 7770, 27954, 21511, 30.08, 1.51),
('A009275', '신원우', 58500, 6100, 11.64, 22565, 58500, 58800, 755, 523, -1950.0, 0),
('A138040', '메리츠금융지주', 19050, 1450, 8.24, 712573, 19000, 19050, 16695, 32307, 4.46, 16.06),
('A000225', '유유제약1우', 14650, 1100, 8.12, 544304, 14600, 14650, 10712, 11443, -406.94, 0),
('A025540', '한국단자', 104000, 7800, 8.11, 166254, 103500, 104000, 4756, 17671, 15.77, 7.84),
('A002140', '고려산업', 4760, 355, 8.06, 3461390, 4755, 4760, 15285, 168996, 14.78, 6.91),
('A334700', 'KBSTAR 팔라듐선물인버스(H)', 4710, 350, 8.03, 11585, 4710, 4730, 17714, 24939, 0, 0),
('A006345', '대원전선우', 7000, 520, 8.02, 506903, 6980, 7000, 9420, 37801, 170.73, 0),
('A030790', '비케이탑스', 9660, 670, 7.45, 459975